# This is a small tutorial for deploying pytorch modules into C++

There are two methods in which we can attain this objective:

* Tracing 
* Annotations

![](Resources/diagram_pytorch_to_c++.jpg)

** Steps to deploy the pytorch model into C++ **

* STEP 1: CONVERT THE MODEL INTO A TORCH SCRIPT
* STEP 2: SERIALIZING YOUR SCRIPT MODULE TO A FILE
* STEP 3: LOADING YOUR SCRIPT MODULE IN C++
* STEP 4: EXECUTING THE SCRIPT MODULE IN C++


## STEP 1: CONVERT THE MODEL INTO A TORCH SCRIPT

## Tracing

tracing is a mechanism in which the structure of the model is captured by evaluating it once using example inputs, and recording the flow of those inputs through the model. This is suitable for models that make limited use of control flow. 

In [1]:
import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
'''jit stands for just in time comiler whick keeps track of all the steps in your model
   to deploy them to c++ afterward'''
traced_script_module = torch.jit.trace(model, example) 

In [2]:
## The traced ScriptModule can now be evaluated identically to a regular PyTorch module:
traced_script_module 

TracedModule[ResNet](
  (conv1): TracedModule[Conv2d]()
  (bn1): TracedModule[BatchNorm2d]()
  (relu): TracedModule[ReLU]()
  (maxpool): TracedModule[MaxPool2d]()
  (layer1): TracedModule[Sequential](
    (0): TracedModule[BasicBlock](
      (conv1): TracedModule[Conv2d]()
      (bn1): TracedModule[BatchNorm2d]()
      (relu): TracedModule[ReLU]()
      (conv2): TracedModule[Conv2d]()
      (bn2): TracedModule[BatchNorm2d]()
    )
    (1): TracedModule[BasicBlock](
      (conv1): TracedModule[Conv2d]()
      (bn1): TracedModule[BatchNorm2d]()
      (relu): TracedModule[ReLU]()
      (conv2): TracedModule[Conv2d]()
      (bn2): TracedModule[BatchNorm2d]()
    )
  )
  (layer2): TracedModule[Sequential](
    (0): TracedModule[BasicBlock](
      (conv1): TracedModule[Conv2d]()
      (bn1): TracedModule[BatchNorm2d]()
      (relu): TracedModule[ReLU]()
      (conv2): TracedModule[Conv2d]()
      (bn2): TracedModule[BatchNorm2d]()
      (downsample): TracedModule[Sequential](
        (0): Tr

## Annotations

This method makes annotations of your model that inform the Torch Script compiler that it may directly parse and compile your model code, subject to the constraints imposed by the Torch Script language.

To observe the changes we will depict both cases where we use pytorch for prototyping (**python**) and for production (**torch script**)

### Prototyping

This is the normal way you define your model in pytorch

In [3]:
import torch

class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if int(input.sum()) > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output
# print model
model = MyModule(5,10)
print(model)

MyModule()


**NOTE:** You could clearly see here that tracing is not suitable for this model because we have a if statement in the forward function, which means that the model depends on the input, and since **tracing** tries out random inputs it might not get all the flow from the model 

### Production deployment

This is the way to follow when we want to deploy the same model into a torch script for production purposes

In [4]:
##just 2 lines change
import torch

class MyModule(torch.jit.ScriptModule): # 1st line modification
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    @torch.jit.script_method # 2nd line addition of an annotation, hence the name of the method 
    def forward(self, input):
        if int(input.sum()) > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

my_script_module = MyModule(5, 10)
print(my_script_module)

MyModule()


## STEP 2: SERIALIZING YOUR SCRIPT MODULE TO A FILE

Serializing is the process of translating the data structure to a format in which the model can be stored. If you want a more detailed explanation on serialization, please refer to this [link](https://en.wikipedia.org/wiki/Serialization)

In [5]:
## After training the model we simply do:
traced_script_module.save("model.pt") #this file can now be loaded into C++

## STEP 3: LOADING YOUR SCRIPT MODULE IN C++

From this step to the last one you must follows this [link](https://pytorch.org/tutorials/advanced/cpp_export.html)
since the remainig steps are done in c++